In [2]:
import sys
sys.path.append("..")
import importlib
import numpy as np
import scipy
import matplotlib.pyplot as plt

import system.experimental_september as system
importlib.reload(system)
import system.tunable_full_cosine as tunable_system
importlib.reload(tunable_system)
import utils.solver as solver
importlib.reload(solver)
import utils.expectation_values as expect
importlib.reload(expect)

pass

In [3]:
E_j = system.E_j
E_c = system.E_c
# initial_guess from truncated power series reversal to full cosine

tunable_system.setup(E_c, E_j)

def energy_gap(level=0):
    return tunable_system.energies[level+1] - tunable_system.energies[level+0]
def anharmonicty():
    return energy_gap(1) - energy_gap(0)

def distance_to_experimental_data():
    return np.abs(system.omega_0 - energy_gap()) + np.abs(system.anharm - anharmonicty())

print("experimental data...")
print(system.omega_0)
print(system.anharm)
print()

print("guess params... ")
print(energy_gap())
print(anharmonicty())

print("distance_to_experimental_data", distance_to_experimental_data())

experimental data...
24.951596996312357
-1.3039494467989794

guess params... 
24.600225881080686
-3.591343091195199
distance_to_experimental_data 2.6387647596278914


In [4]:
def cost(parameters):
    E_c = parameters[0]
    E_j = parameters[1]
    tunable_system.setup(E_c, E_j, _basis_size=_basis_size)
    return distance_to_experimental_data()

ree = scipy.optimize.minimize(cost, [E_c, E_j], method="Nelder-Mead", options={"disp":True, "maxiter": 10000})

Optimization terminated successfully.
         Current function value: 0.000001
         Iterations: 72
         Function evaluations: 135


In [5]:
tunable_system.setup(ree.x[0], ree.x[1])

In [6]:
print("experimental data...")
print(system.omega_0)
print(system.anharm)
print()

print("guess params... ")
print(energy_gap())
print(anharmonicty())

print("distance_to_experimental_data", distance_to_experimental_data())

experimental data...
24.951596996312357
-1.3039494467989794

guess params... 
24.951597002504336
-1.303950226078328
distance_to_experimental_data 7.854713270916136e-07


In [7]:
print("optimized: ", ree.x)
print("experimental data:", [system.E_c, system.E_j])

optimized:  [ 1.15849102 73.88873718]
experimental data: [2.607898893597959, 36.40506422481128]


# Finding the right params for much higher basis size

In [15]:
tunable_system.setup(ree.x[0], ree.x[1], _basis_size=1000)
ree = scipy.optimize.minimize(cost, [E_c, E_j], method="Nelder-Mead", options={"disp":True, "maxiter": 10000})

Optimization terminated successfully.
         Current function value: 0.000001
         Iterations: 72
         Function evaluations: 135


In [16]:
ree

 final_simplex: (array([[ 1.15849102, 73.88873718],
       [ 1.15848979, 73.88881041],
       [ 1.15849072, 73.88874937]]), array([7.85471327e-07, 1.24868182e-06, 1.28651496e-06]))
           fun: 7.854713270916136e-07
       message: 'Optimization terminated successfully.'
          nfev: 135
           nit: 72
        status: 0
       success: True
             x: array([ 1.15849102, 73.88873718])